## Unite all data

In [1]:
import pandas as pd
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import nltk

positives=pd.read_csv('positive_combined.csv')
cases=pd.read_csv('case_files.csv')

positives['class']=positives['class'].str.replace(' ','')
cases_clean=pd.DataFrame(data={'class':cases['Case Type'],'complain':cases['Case Summary']})

df=pd.concat([positives,cases_clean], axis= 0)
df['class']=df['class'].apply(lambda x: x[0].upper()+x[1:])
df['class'].unique()

array(['Notrelated', 'Labor', 'Minor', 'Adult'], dtype=object)

## Text preprocessing

In [2]:
nltk.download('punkt')
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

[nltk_data] Downloading package punkt to /Users/maria/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

